<a href="https://colab.research.google.com/github/aljoharas/datascience/blob/main/spotifyproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA

In [1]:
!pip install spotipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 7.2 MB/s eta 0:00:00


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

CLIENT_ID = "091a90ce07be44b2b4bb6f6fcf08fd31" #from our developer spotify acc
CLIENT_SECRET = "3333569ba9d44fc48d0b4d1e422db526"  #from our developer spotify acc


auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm


In [4]:
import pandas as pd

url = "/content/regional-global-weekly-2025-09-18.csv"
df = pd.read_csv(url)


print(df.head())

artists = df['artist_names'].unique()
print(f"✅ Found {len(artists)} unique artists")


   rank                                   uri  \
0     1  spotify:track:1CPZ5BxNNd0n0nF4Orb9JS   
1     2  spotify:track:7qjZnBKE73H4Oxkopwulqe   
2     3  spotify:track:2RkZ5LkEzeHGRsmDqKwmaJ   
3     4  spotify:track:02sy7FAs8dkDNYsHp4Ul3f   
4     5  spotify:track:42VUCXerQ5qTr4Qp6PhKo4   

                                        artist_names       track_name  \
0  HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...           Golden   
1                                              sombr  back to friends   
2                                        Alex Warren         Ordinary   
3  Saja Boys, Andrew Choi, Neckwav, Danny Chung, ...         Soda Pop   
4                                  Sabrina Carpenter            Tears   

                source  peak_rank  previous_rank  weeks_on_chart   streams  
0  K-Pop Demon Hunters          1              1              13  52790939  
1       Warner Records          1              2              26  42212664  
2     Atlantic Records          1  

In [5]:
artist_data = []
for artist_name in artists[:50]:
    result = sp.search(q=f"artist:{artist_name}", type="artist", limit=1)
    if result['artists']['items']:
        a = result['artists']['items'][0]
        artist_data.append({
            "name": a['name'],
            "id": a['id'],
            "genres": a['genres'],
            "popularity": a['popularity']
        })

df_artists = pd.DataFrame(artist_data)
print(df_artists.head())
#tesssst


                name                      id      genres  popularity
0              sombr  4G9NDjRyZFDlJKMRL8hx3S          []          90
1        Alex Warren  0fTSzq9jAh4c36UVb4V7CB          []          89
2  Sabrina Carpenter  74KM79TiuVKeVCqs8QtB0B       [pop]          97
3     Artist Unknown  6gjAbs229sNHhSskJpQ2mG     [opera]          24
4        Olivia Dean  00x1fYSGhdqScXBRpSj3DW  [pop soul]          86


In [20]:

# ---- Load Global CSV ----
df_global = pd.read_csv("/content/regional-global-weekly-2025-09-18.csv")

# ---- Collect artist info (Global) ----
artist_data_global = []
count = 0

for artist_name in artists_global:
    if count >= 200:
        break
    result = sp.search(q=f"artist:{artist_name}", type="artist", limit=1)
    if result['artists']['items']:
        a = result['artists']['items'][0]
        if not a['genres']:
            continue
        artist_data_global.append({
            "name": a['name'],
            "id": a['id'],
            "genres": a['genres'],
            "popularity": a['popularity']
        })
        count += 1

# ---- Create DataFrame (Global) ----
df_artists_global = pd.DataFrame(artist_data_global).head(200)
print(f"✅ Final Global dataset: {len(df_artists_global)} artists with non-empty genres")
display(df_artists_sa.head())

✅ Final Global dataset: 45 artists with non-empty genres


,name,id,genres,popularity
0,Artist Unknown,6gjAbs229sNHhSskJpQ2mG,[opera],24
1,Sabah Mahmod,6DPTJXd1omQ1oTiR1Vy9CZ,[khaleeji],48
2,The Marías,2sSGPbdZJkaSE2AbcGOACx,[bedroom pop],87
3,TUL8TE,5QKGejJMncXUNUb9pUFbEf,"[egyptian hip hop, arabic hip hop, egyptian po...",68
4,Radiohead,4Z8W4fKeB5YxbusRsdQVPb,"[art rock, alternative rock]",89


In [19]:
# ---- Load Saudi CSV ----
df_sa = pd.read_csv("/content/regional-sa-weekly-2025-09-18.csv")

# ---- Collect artist info (Saudi) ----
artist_data_sa = []
count = 0

for artist_name in artists_sa:
    if count >= 200:
        break
    result = sp.search(q=f"artist:{artist_name}", type="artist", limit=1)
    if result['artists']['items']:
        a = result['artists']['items'][0]
        if not a['genres']:
            continue
        artist_data_sa.append({
            "name": a['name'],
            "id": a['id'],
            "genres": a['genres'],
            "popularity": a['popularity']
        })
        count += 1

# ---- Create DataFrame (Saudi) ----
df_artists_sa = pd.DataFrame(artist_data_sa).head(200)
print(f"✅ Final Saudi dataset: {len(df_artists_sa)} artists with non-empty genres\n")
display(df_artists_sa.head())

✅ Final Saudi dataset: 75 artists with non-empty genres



,name,id,genres,popularity
0,Artist Unknown,6gjAbs229sNHhSskJpQ2mG,[opera],24
1,Sabah Mahmod,6DPTJXd1omQ1oTiR1Vy9CZ,[khaleeji],48
2,The Marías,2sSGPbdZJkaSE2AbcGOACx,[bedroom pop],87
3,TUL8TE,5QKGejJMncXUNUb9pUFbEf,"[egyptian hip hop, arabic hip hop, egyptian po...",68
4,Radiohead,4Z8W4fKeB5YxbusRsdQVPb,"[art rock, alternative rock]",89
